In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("/kaggle/input/travel-insurance/travel insurance.csv")
data.head()

In [ ]:
data.columns

In [ ]:
data.Claim.unique()

In [ ]:
data.Claim.value_counts()

In [ ]:
data.isnull().sum()

In [ ]:
data.Gender=data.Gender.fillna("default")
data.isnull().sum()

In [ ]:
data

In [ ]:
import seaborn as sns
sns.distplot(data.Age)


In [ ]:
sns.pairplot(data,hue="Claim")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
sns.heatmap(data.corr(),annot=True)

In [ ]:
num_cols=data.select_dtypes(include=['int',float])
num_cols

In [ ]:
data.select_dtypes(exclude=('int',float))

In [ ]:
sns.boxplot(data.Age)

In [ ]:
x=data.drop(columns=['Claim'])
y=data[['Claim']]
x.head()

In [ ]:
y.head()

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=50)
x_train.head()

In [ ]:
data.Gender.unique()

In [ ]:
le=LabelEncoder()
y[['Claim']]=pd.DataFrame(le.fit_transform(y[['Claim']]))
y.head()

In [ ]:
nom_col=[0,1,3,5,8]
ord_col=[2]

In [ ]:
x_train

In [ ]:
pd.get_dummies(data.loc[nom_col],drop_first=True)

In [ ]:
trans_data=make_column_transformer((OneHotEncoder(sparse=False,handle_unknown='ignore'),[0,1,3,5,8]),
                                   (OrdinalEncoder(),[2]),
                                   remainder='passthrough')

In [ ]:
xp = trans_data.fit_transform(x_train)
xp

In [ ]:
model=LogisticRegression(solver='liblinear')

In [ ]:
set_config(display='diagram')
pipe=make_pipeline(trans_data,model)
pipe

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
pred=pipe.predict(x_test)

In [ ]:
accuracy_score(pred,y_test)*100

In [ ]:
trans_data.fit_transform(x)

In [ ]:
model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=5)

In [ ]:
set_config(display='diagram')
pipe=make_pipeline(trans_data,model)
pipe

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
pred=pipe.predict(x_test)

In [ ]:
accuracy_score(pred,y_test)*100

In [ ]:
trans_data.fit_transform(x_train)

In [ ]:
model=KNeighborsClassifier(n_neighbors=5)

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold,cross_val_score
S=StratifiedKFold(n_splits=5)
np.mean(cross_val_score(pipe,x,y,cv=5,scoring='accuracy'))*100

In [ ]:
sns.countplot(data.Claim)

In [ ]:
data.Claim.value_counts()

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
under=RandomUnderSampler()
x_u,y_u=under.fit_resample(x,y)
sns.countplot(y_u.Claim)

In [ ]:
y_u.Claim.value_counts()

In [ ]:
from imblearn.over_sampling import RandomOverSampler
over=RandomOverSampler()
x_o,y_o=over.fit_resample(x,y)
sns.countplot(y_o.Claim)

In [ ]:
y_o.Claim.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE()

x_s,y_s=smote.fit_resample(trans_data.fit_transform(x),y)
sns.countplot(y_s.Claim)

In [ ]:
x_s

In [ ]:
y_s

In [ ]:
from imblearn.pipeline import make_pipeline
p=make_pipeline(trans_data,model)
s=StratifiedKFold(n_splits=5)
np.mean(cross_val_score(p,trans_data.fit_transform(x_s),(y_s),cv=5,scoring="accuracy")*100)
